RandomForest는 tree기반 모델로, DecisionTree를 여러 개 생성하여 그들의 평균으로 예측의 성능을 높이는 Ensemble Machine Learning 모델입니다. <br>
RandomForest에 대해 더 자세히 알고싶으시다면 아래의 링크를 참고해주시기 바랍니다. <br>

[Lv2 모델링 1/2 python 파이썬 랜덤포레스트 개념, 선언](https://dacon.io/competitions/open/235698/talkboard/403632) <br>

# Library load

In [15]:
import random
import os

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

## 경고 무시
import warnings
warnings.filterwarnings('ignore')

## 결과확인을 용이하게 하는 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 변수 고정
변수를 고정하는 방법에는 다양한 방법(dictionary 내에 정의, 하나의 전역변수로 정의 등)이 있지만,
이번 베이스라인에서는 class를 사용하여 변수를 고정하는 방법을 사용해보도록 하겠습니다.

In [16]:
class CFG:
    user_seed = 42
    target = 'Prospect'

# Seed 고정

In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG.user_seed)

# Load data

In [18]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

## isnull

In [19]:
train.isnull().sum().sum()
## 결측이 확인되지 않습니다.

0

# Data preprocessing

## LabelEncoding
- qualitative to quantitative

In [20]:
qual_col = ['Position', 'PreferredFoot', 'AttackingWorkRate', 'DefensiveWorkRate']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    ## Label Encoder가 Test 데이터로부터 Fitting되는 것은 Data Leakage이므로, Test 데이터에는 Train 데이터로 Fitting된 Label Encoder로부터 transform만 수행되어야 합니다.
    test[i] = le.transform(test[i])
print('Done.')

Done.


## scaling
기본적인 모델링 프로세스대로라면 전처리 단계에서 수치형 변수에 대해 scale을 맞춰주는 scaling을 진행해야 합니다. <br>
하지만 이번 베이스라인에서는 tree 기반 모델을 사용할 것이기 때문에 해당 단계는 넘어가도 무관합니다. <br>

scaling에 대해 더 자세히 알고싶으시다면 아래의 링크를 참고해주시기 바랍니다. <br>
[Lv3 전처리 3/4 python 파이썬 수치형 데이터 정규화](https://dacon.io/competitions/open/235698/talkboard/403825?page=1&dtype=recent)

# X, y split

In [21]:
X_train = train.drop(['ID', CFG.target], axis=1)
y_train = train[CFG.target]

# Classification model fit

In [22]:
model = RandomForestClassifier(random_state = CFG.user_seed)
## train
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# Inference

In [23]:
X_test = test.drop(['ID'], axis=1)

## pred
pred = model.predict(X_test)

# Submit

In [10]:
submit = pd.read_csv('./sample_submission.csv')

In [11]:
submit[CFG.target] = pred

In [12]:
submit.to_csv('submit.csv', index = False)